Setup Environment

In [26]:
# Clear the R environment and set working directory
rm(list = ls())
setwd("/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters")
path <- getwd()
path 
options(warn=-1)

[1] "/Users/jan/Dropbox/UP_EPQM/2222/MA/powerlinemonsters"

Load packages

In [89]:
# Install and load additional packages needed
#install.packages("stringr")
library(did)
library(rio)
library(ggplot2)
library(ggpubr)
library(hash)
.libPaths()


The downloaded binary packages are in
	/var/folders/wm/mtrphj0s0msgrmyshf3hgk740000gn/T//RtmpCM358d/downloaded_packages


[1] "/Library/Frameworks/R.framework/Versions/4.2-arm64/Resources/library"

Setup TeX Code to integrate figures

In [113]:
fig_tex <- r"(\renewcommand{\thefigure}{%s} \begin{figure} \centering \includegraphics[width=1\textwidth]{%s} \captionlistentry{%s} \label{fig:%s} \end{figure})"
figs_tex <- list()

DiD for BTW no controls

In [114]:
# read in the data set
btw <- import(sprintf("%s/data/btw_treat.csv", path))

In [119]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
treatments <- keys(treatments_dict)
results  <- list()
# set up Fig Number
n <- 1
for(party in parties){
  figures  <- list()
  for(treatment in treatments){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ 1,
                  data = btw,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(format(result$n, big.mark=","), big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
  }
  # Combine Figures for all treatments
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
  # get legend
  result_fig <- ggdid(result) +
              theme(
              legend.background = element_rect(fill='transparent', color='transparent'),
              legend.box.background = element_rect(fill='transparent', color='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
  leg <- get_legend(result_fig)
  # add legend
  arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
  # Annotate final figure
  title <- sprintf("BTW - Effect on %s's vote share \n no controls", party)
  final_fig <- annotate_figure(arranged_fig_leg,
                  top = text_grob(title, face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression \n Standard Errors clustered on the municipality level.', size = 10))
  # Add Figure number
  number <- sprintf("R%s", n)
  final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: ", number), face = "bold", size = 10))
  # Save
  filename <- sprintf('%s_%s.png', n, party)
  ggsave(filename, plot = final_fig, path = sprintf('%s/figures/R/BTW/pooled', path), units = 'cm', width = 21, height = 21, dpi="print")
  # Add Reference to figs list
  figs_tex[[length(figs_tex)+1]] <- sprintf(fig_tex, number, filename, title, number)
  print(party)
  n <- n + 1
}

[1] "Union"
[1] "SPD"
[1] "FDP"
[1] "Linke"
[1] "Grüne"
[1] "Andere"


In [120]:
figs_tex

[[1]]
[1] "\\renewcommand{\\thefigure}{R1} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{1_Union.png} \\captionlistentry{Union} \\label{fig:R1} \\end{figure}"

[[2]]
[1] "\\renewcommand{\\thefigure}{R1} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{1_Union.png} \\captionlistentry{Union} \\label{fig:R1} \\end{figure}"

[[3]]
[1] "\\renewcommand{\\thefigure}{R1} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{1_Union.png} \\captionlistentry{Union} \\label{fig:R1} \\end{figure}"

[[4]]
[1] "\\renewcommand{\\thefigure}{R1} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{1_Union.png} \\captionlistentry{Union} \\label{fig:R1} \\end{figure}"

[[5]]
[1] "\\renewcommand{\\thefigure}{R1} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{1_Union.png} \\captionlistentry{Union} \\label{fig:R1} \\end{figure}"

[[6]]
[1] "\\renewcommand{\\thefigure}{R1} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{1_Union.png} \\captionlistentry{Union} \\label{fig:R1} \\end{figure}"

[[7]]
[1] "\\renewcommand{\\thefigure}{Fig.: R1} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{1_Union.png} \\captionlistentry{BTW - Effect on Union's vote share \n no controls} \\label{fig:Fig.: R1} \\end{figure}"

[[8]]
[1] "\\renewcommand{\\thefigure}{Fig.: R2} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{2_SPD.png} \\captionlistentry{BTW - Effect on SPD's vote share \n no controls} \\label{fig:Fig.: R2} \\end{figure}"

[[9]]
[1] "\\renewcommand{\\thefigure}{Fig.: R3} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{3_FDP.png} \\captionlistentry{BTW - Effect on FDP's vote share \n no controls} \\label{fig:Fig.: R3} \\end{figure}"

[[10]]
[1] "\\renewcommand{\\thefigure}{Fig.: R4} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{4_Linke.png} \\captionlistentry{BTW - Effect on Linke's vote share \n no controls} \\label{fig:Fig.: R4} \\end{figure}"

[[11]]
[1] "\\renewcommand{\\thefigure}{Fig.: R5} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{5_Grüne.png} \\captionlistentry{BTW - Effect on Grüne's vote share \n no controls} \\label{fig:Fig.: R5} \\end{figure}"

[[12]]
[1] "\\renewcommand{\\thefigure}{Fig.: R6} \\begin{figure} \\centering \\includegraphics[width=1\\textwidth]{6_Andere.png} \\captionlistentry{BTW - Effect on Andere's vote share \n no controls} \\label{fig:Fig.: R6} \\end{figure}"

DiD for BTW with controls

In [30]:
btw_c <- import(sprintf("%s/data/btw_control.csv", path))

In [31]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
treatments <- keys(treatments_dict)
results  <- list()
n <- 7
for(party in parties){
  figures  <- list()
  for(treatment in treatments){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ east + south + pop_density + unemployed + female + avg_age,
                  data = btw_c,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
  }
  # Combine Figures for all treatments
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
  # get legend
  result_fig <- ggdid(result) +
              theme(
              legend.background = element_rect(fill='transparent', color='transparent'),
              legend.box.background = element_rect(fill='transparent', color='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
  leg <- get_legend(result_fig)
  # add legend
  arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
  # Annotate final figure
  final_fig <- annotate_figure(arranged_fig_leg,
                  top = text_grob(sprintf("BTW - Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression. \n Standard Errors clustered on the municipality level. \n Included Controls: Population Density, Share of Unemployed, Share of Females, Average Age, \n Dummies for East and South Germany.', size = 10))
  # Add Figure number
  final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
  # Save
  ggsave(sprintf('%s_%s.png', n, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/control', path), units = 'cm', width = 21, height = 21, dpi="print")
  print(party)
  n <- n + 1
}

[1] "Union"
[1] "SPD"
[1] "FDP"
[1] "Linke"
[1] "Grüne"
[1] "Andere"


DiD for LTW no controls by state

In [32]:
# read in the data set
ltw <- import(sprintf("%s/data/ltw_treat.csv", path))

In [34]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
states_dict <- hash('NI'='Lower Saxony', 'NW'='North-Rhine Westphalia', 'HE'='Hesse', 'BY'='Bavaria')
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
n <- 14
for (state in states){
  for(party in parties){
    figures  <- list()
    reg_data <- subset(ltw, Land == state)
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ 1,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'notyettreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("LTW %s - Effect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Not-yet Treated. Estimation Method: Regression \n Standard Errors clustered on the municipality level.', size = 10),)
    # Add Figure number
    final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
    # Save
    ggsave(sprintf('%s_%s_%s.png', n, state, party), plot = final_fig, path = sprintf('%s/figures/R/LTW/by_state', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(sprintf('%s: %s', state, party))
    n <- n + 2
  }
}

[1] "BY: Union"
[1] "BY: SPD"
[1] "BY: FDP"
[1] "BY: Linke"
[1] "BY: Grüne"
[1] "BY: Andere"
[1] "HE: Union"
[1] "HE: SPD"
[1] "HE: FDP"
[1] "HE: Linke"
[1] "HE: Grüne"
[1] "HE: Andere"
[1] "NI: Union"
[1] "NI: SPD"
[1] "NI: FDP"
[1] "NI: Linke"
[1] "NI: Grüne"
[1] "NI: Andere"
[1] "NW: Union"
[1] "NW: SPD"
[1] "NW: FDP"
[1] "NW: Linke"
[1] "NW: Grüne"
[1] "NW: Andere"


DiD for LTW controls by state

In [36]:
ltw_c <- import(sprintf("%s/data/ltw_control.csv", path))

In [37]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
states_dict <- hash('NI'='Lower Saxony', 'NW'='North-Rhine Westphalia', 'HE'='Hesse', 'BY'='Bavaria')
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
n <- 15
for (state in states){
  for(party in parties){
    figures  <- list()
    reg_data <- subset(ltw_c, Land == state)
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ pop_density + unemployed + avg_age + female,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'notyettreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("LTW %s - Effect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Not-yet Treated. Estimation Method: Regression. \n Standard Errors clustered on the municipality level. \n Included Controls: Population Density, Share of Unemployed, Share of Females, Average Age.', size = 10))
    # Add Figure number
    final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
    # Save
    ggsave(sprintf('%s_%s_%s.png', n, state, party), plot = final_fig, path = sprintf('%s/figures/R/LTW/by_state_controls', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    print(sprintf('%s: %s', state, party))
    n <- n + 2
  }
}

[1] "BY: Union"
[1] "BY: SPD"
[1] "BY: FDP"
[1] "BY: Linke"
[1] "BY: Grüne"
[1] "BY: Andere"
[1] "HE: Union"
[1] "HE: SPD"
[1] "HE: FDP"
[1] "HE: Linke"
[1] "HE: Grüne"
[1] "HE: Andere"
[1] "NI: Union"
[1] "NI: SPD"
[1] "NI: FDP"
[1] "NI: Linke"
[1] "NI: Grüne"
[1] "NI: Andere"
[1] "NW: Union"
[1] "NW: SPD"
[1] "NW: FDP"
[1] "NW: Linke"
[1] "NW: Grüne"
[1] "NW: Andere"


Robustness 1: BTW control without postal votes

In [106]:
btw_np <- import(sprintf("%s/data/btw_control_nopostal.csv", path))

In [109]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
treatments <- keys(treatments_dict)
results  <- list()
n <- 62
for(party in parties){
  figures  <- list()
  for(treatment in treatments){
    result <- att_gt(yname = party,
                  gname = treatment,
                  idname = 'AGS',
                  panel = TRUE,
                  tname = 'year',
                  xformla = ~ east + south + pop_density + unemployed + female + avg_age,
                  data = btw_c,
                  est_method = 'reg',
                  anticipation = 0,
                  control_group = 'nevertreated',
                  clustervars = c('AGS'),
                  bstrap = TRUE,
                  cband = TRUE,
                  allow_unbalanced_panel = TRUE,
                  
    )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              #ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
    labs(caption = cap) + 
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
  }
  # Combine Figures for all treatments
  arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
  # get legend
  result_fig <- ggdid(result) +
              theme(
              legend.background = element_rect(fill='transparent', color='transparent'),
              legend.box.background = element_rect(fill='transparent', color='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
  leg <- get_legend(result_fig)
  # add legend
  arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
  # Annotate final figure
  final_fig <- annotate_figure(arranged_fig_leg,
                  top = text_grob(sprintf("BTW - Effect on %s's vote share", party), face = "bold", size = 14),
                  bottom = text_grob('Control Group: Never Treated. Estimation Method: Regression. \n Standard Errors clustered on the municipality level. \n Included Controls: Population Density, Share of Unemployed, Share of Females, Average Age, \n Dummies for East and South Germany.', size = 10))
  # Add Figure number
  final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
  # Save
  ggsave(sprintf('%s_%s.png', n, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/no_postal', path), units = 'cm', width = 21, height = 21, dpi="print")
  print(party)
  n <- n + 1
}

[1] "Union"
[1] "SPD"
[1] "FDP"
[1] "Linke"
[1] "Grüne"
[1] "Andere"


DiD for BTW with controls by state

In [ ]:
btw_c <- import(sprintf("%s/data/btw_control.csv", path))

In [45]:
parties <- c('Union', 'SPD', 'FDP', 'Linke', 'Grüne', 'Andere')
treatments_dict <- hash('treatment_0'='Direct Line', 'treatment_15'='Within 0-15 km', 'treatment_30'='Within 15-30 km', 'treatment_50'='Within 30-50 km')
states_dict <- hash('SH'='Schleswig-Holstein', 'NI'='Lower Saxony', 'NW'='North-Rhine Westphalia', 'BW'='Baden-Württemberg', 
'BY'='Bavaria', 'BB'='Brandenburg', 'MV'='Mecklenburg-Vorpommern', 'ST'='Saxony-Anhalt', 'TH'='Thuringia', 'RP'='Rhineland-Palatine') # all states - those never treated
treatments <- keys(treatments_dict)
states <- keys(states_dict)
results  <- list()
n <- 62
for (state in states){
  for(party in parties){
    print(sprintf('%s: %s', state, party))
    reg_data <- subset(btw_np, Land == state)
    figures  <- list()
    for(treatment in treatments){
      result <- att_gt(yname = party,
                    gname = treatment,
                    idname = 'AGS',
                    panel = TRUE,
                    tname = 'year',
                    xformla = ~ pop_density + unemployed + female + avg_age,
                    data = reg_data,
                    est_method = 'reg',
                    anticipation = 0,
                    control_group = 'notyettreated',
                    clustervars = c('AGS'),
                    bstrap = TRUE,
                    cband = TRUE,
                    allow_unbalanced_panel = TRUE,
                    
      )
    results[[length(results)+1]] <- result
    # calculate ATT
    att <- aggte(result, type = "group", bstrap = TRUE, clustervars = c('AGS'))
    # plot results
    result_fig <- ggdid(result, 
              ylim = c(floor(min(result$att - result$se * 2.345 - 1)), ceiling(max(result$att + result$se * 2.345 + 1))),
              ncol = 3, ax_text_size = 8, grtitle='', title_size = 10, legend=F) +
              labs(caption = sprintf('Overall ATT (SE): %.3f (%.3f) \n Wald-p: %.3f; Municipalities: %s \n' ,
              att$overall.att, att$overall.se, result$Wpval, format(result$n, big.mark=","))) +
              theme(plot.caption = element_text(hjust=0.5, size=8),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Get caption for ATT plot
    cap = ''
      for(i in 1:length(att$DIDparams$glist)) {
        cap <- sprintf('%s ATT (SE): %.3f (%.3f)\n%s', att$DIDparams$glist[[i]], att$att.egt[[i]], att$se.egt[[i]], cap)
      }
    # Plot ATT
    att_fig <- ggdid(att, legend=F, x_lab='', ax_text_size = 8, ylab='', title='ATT by group', title_size = 10) + 
              labs(caption = cap) +
              theme(plot.caption = element_text(hjust=0.5, size=8), 
              axis.title.x = element_blank(),
              axis.text.y = element_text(angle=90, size=6, hjust=0.5),
              panel.background = element_rect(fill='transparent'),
              plot.background = element_rect(fill='transparent', color=NA),
              legend.background = element_rect(fill='transparent'),
              legend.box.background = element_rect(fill='transparent'),
              plot.margin=unit(c(0,0,0,0),"cm"))
    # Combine and annotate Group and ATT plot
    # Get number of results plots
    nr <- length(unique(result_fig$data$grtitle))
    combined_fig <- ggarrange(result_fig, att_fig, widths = c(nr, 1)) + 
                    theme(plot.margin=unit(c(0,0,0,0),"cm"))
    combined_fig_anno <- annotate_figure(combined_fig,
                  top = text_grob(sprintf('Treatment: %s', treatments_dict[[treatment]])))
    figures[[length(figures)+1]] <- combined_fig_anno
    }
    # Combine Figures for all treatments
    arranged_fig <- ggarrange(plotlist=figures, nrow = 4, ncol = 1, common.legend = TRUE)
    # get legend
    result_fig <- ggdid(result) +
                theme(
                legend.background = element_rect(fill='transparent', color='transparent'),
                legend.box.background = element_rect(fill='transparent', color='transparent'),
                plot.margin=unit(c(0,0,0,0),"cm"))
    leg <- get_legend(result_fig)
    # add legend
    arranged_fig_leg <- ggarrange(arranged_fig, leg, nrow = 2, ncol = 1, heights = c(10, 1))
    # Annotate final figure
    final_fig <- annotate_figure(arranged_fig_leg,
                    top = text_grob(sprintf("BTW in %s - Effect on %s's vote share", states_dict[[state]], party), face = "bold", size = 14),
                    bottom = text_grob('Control Group: Not-yet Treated. Estimation Method: Regression. \n Standard Errors clustered on the municipality level. \n Included Controls: Population Density, Share of Unemployed, Share of Females, Average Age.', size = 10))
    # Add Figure number
    final_fig <- annotate_figure(final_fig, top = text_grob(sprintf("Fig.: R%s", n), face = "bold", size = 10))
    # Save
    ggsave(sprintf('%s_%s_%s.png', n, state, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/by_state/all', path), units = 'cm', width = 21, height = 29.7, dpi="print")
    ggsave(sprintf('%s_%s_%s.png', n, state, party), plot = final_fig, path = sprintf('%s/figures/R/BTW/by_state/%s', path, party), units = 'cm', width = 21, height = 29.7, dpi="print")
    n <- n + 1
  }
}

[1] "BB: Union"
[1] "BB: SPD"
[1] "BB: FDP"
[1] "BB: Linke"
[1] "BB: Grüne"
[1] "BB: Andere"
[1] "BW: Union"
[1] "BW: SPD"
[1] "BW: FDP"
[1] "BW: Linke"
[1] "BW: Grüne"
[1] "BW: Andere"
[1] "BY: Union"
[1] "BY: SPD"
[1] "BY: FDP"
[1] "BY: Linke"
[1] "BY: Grüne"
[1] "BY: Andere"
[1] "MV: Union"
[1] "MV: SPD"
[1] "MV: FDP"
[1] "MV: Linke"
[1] "MV: Grüne"
[1] "MV: Andere"
[1] "NI: Union"
[1] "NI: SPD"
[1] "NI: FDP"
[1] "NI: Linke"
[1] "NI: Grüne"
[1] "NI: Andere"
[1] "NW: Union"
[1] "NW: SPD"
[1] "NW: FDP"
[1] "NW: Linke"
[1] "NW: Grüne"
[1] "NW: Andere"
[1] "RP: Union"
[1] "RP: SPD"
[1] "RP: FDP"
[1] "RP: Linke"
[1] "RP: Grüne"
[1] "RP: Andere"
[1] "SH: Union"
[1] "SH: SPD"
[1] "SH: FDP"
[1] "SH: Linke"
[1] "SH: Grüne"
[1] "SH: Andere"
[1] "ST: Union"
[1] "ST: SPD"
[1] "ST: FDP"
[1] "ST: Linke"
[1] "ST: Grüne"
[1] "ST: Andere"
[1] "TH: Union"
[1] "TH: SPD"
[1] "TH: FDP"
[1] "TH: Linke"
[1] "TH: Grüne"
[1] "TH: Andere"
